In [1]:
import sys
sys.path.append( '../' )

In [2]:
import numpy as np
import cs_vqe as c
import ast
import os
import scipy as sp
from copy import deepcopy

In [3]:
from openfermion import qubit_operator_sparse
import quchem.Misc_functions.conversion_scripts as conv_scr
from openfermion.ops import QubitOperator

In [4]:
# with open("hamiltonians.txt", 'r') as input_file:
#     hamiltonians = ast.literal_eval(input_file.read())
    
cwd = os.getcwd()
working_dir = os.path.dirname(cwd)
data_dir = os.path.join(working_dir, 'data')
data_hamiltonians_file = os.path.join(data_dir, 'hamiltonians.txt')


with open(data_hamiltonians_file, 'r') as input_file:
    hamiltonians = ast.literal_eval(input_file.read())

In [5]:
for key in hamiltonians.keys():
    print(f"{key: <25}     n_qubits:  {hamiltonians[key][1]:<5.0f}")

H2-S1_STO-3G_singlet          n_qubits:  18   
C1-O1_STO-3G_singlet          n_qubits:  16   
H1-Cl1_STO-3G_singlet         n_qubits:  16   
H1-Na1_STO-3G_singlet         n_qubits:  16   
H2-Mg1_STO-3G_singlet         n_qubits:  17   
H1-F1_3-21G_singlet           n_qubits:  18   
H1-Li1_3-21G_singlet          n_qubits:  18   
Be1_STO-3G_singlet            n_qubits:  5    
H1-F1_STO-3G_singlet          n_qubits:  8    
H1-Li1_STO-3G_singlet         n_qubits:  8    
Ar1_STO-3G_singlet            n_qubits:  13   
F2_STO-3G_singlet             n_qubits:  15   
H1-O1_STO-3G_singlet          n_qubits:  8    
H2-Be1_STO-3G_singlet         n_qubits:  9    
H2-O1_STO-3G_singlet          n_qubits:  10   
H2_3-21G_singlet              n_qubits:  5    
H2_6-31G_singlet              n_qubits:  5    
H3-N1_STO-3G_singlet          n_qubits:  13   
H4-C1_STO-3G_singlet          n_qubits:  14   
Mg1_STO-3G_singlet            n_qubits:  13   
N2_STO-3G_singlet             n_qubits:  15   
Ne1_STO-3G_si

In [6]:
data_csvqe_results_file = os.path.join(data_dir, 'csvqe_results.txt')
with open(data_csvqe_results_file, 'r') as input_file:
    csvqe_results = ast.literal_eval(input_file.read())

for key in csvqe_results.keys():
    print(f"{key: <25}     n_qubits:  {hamiltonians[key][1]:<5.0f}")

H3_STO-3G_singlet_1+          n_qubits:  3    
Be1_STO-3G_singlet            n_qubits:  5    
H2_3-21G_singlet              n_qubits:  5    
H2_6-31G_singlet              n_qubits:  5    
H1-He1_3-21G_singlet_1+       n_qubits:  6    
H1-F1_STO-3G_singlet          n_qubits:  8    
H1-Li1_STO-3G_singlet         n_qubits:  8    
H1-O1_STO-3G_singlet          n_qubits:  8    
H2-Be1_STO-3G_singlet         n_qubits:  9    
H3_3-21G_singlet_1+           n_qubits:  9    
H2-O1_STO-3G_singlet          n_qubits:  10   
H3-N1_STO-3G_singlet          n_qubits:  13   
Mg1_STO-3G_singlet            n_qubits:  13   
H4-C1_STO-3G_singlet          n_qubits:  14   
H4-N1_STO-3G_singlet_1+       n_qubits:  14   
F2_STO-3G_singlet             n_qubits:  15   
N2_STO-3G_singlet             n_qubits:  15   
O2_STO-3G_singlet             n_qubits:  15   
C1-O1_STO-3G_singlet          n_qubits:  16   
H1-Cl1_STO-3G_singlet         n_qubits:  16   
H1-Na1_STO-3G_singlet         n_qubits:  16   
H2-Mg1_STO-3G

In [7]:
# mol_k
# mol_key ='H3_STO-3G_singlet_1+'
mol_key ='H2-Be1_STO-3G_singlet'
# mol_key = 'H1-Li1_STO-3G_singlet'


In [8]:
print(f"n_qubits:  {hamiltonians[mol_key][1]}")

n_qubits:  9


# Get non-contextual H

In [9]:
ham = hamiltonians[mol_key][2]
ham

{'IIIIIIIII': -11.893640687855415,
 'IIIIIIIIZ': 2.2179177068982137,
 'IIIIIIIYY': -0.005675199558538011,
 'IIIIIIIZI': 2.2179177068982137,
 'IIIIIIIZZ': 0.5678633869112119,
 'IIIIIIXIX': -0.050479866172207837,
 'IIIIIIXXI': -0.0009000465964271142,
 'IIIIIIXZX': -0.03270134934370917,
 'IIIIIIYIY': -0.050479866172207837,
 'IIIIIIYZY': -0.03270134934370917,
 'IIIIIIZII': 0.00936892585086291,
 'IIIIIIZIZ': 0.11669843437820623,
 'IIIIIIZZI': 0.12357978573297904,
 'IIIIIXIXI': -0.0017481184367266682,
 'IIIIIXXYY': -0.006881351354772961,
 'IIIIIXYYX': 0.006881351354772961,
 'IIIIIXZXI': -0.03270134934370917,
 'IIIIIXZXZ': -0.050479866172207837,
 'IIIIIYIIY': -0.0009000465964271142,
 'IIIIIYIYI': -0.0017481184367266682,
 'IIIIIYXXY': 0.006881351354772961,
 'IIIIIYYII': -0.019211166510235175,
 'IIIIIYYXX': -0.006881351354772961,
 'IIIIIYYZZ': -0.019211166510235175,
 'IIIIIYZYI': -0.03270134934370917,
 'IIIIIYZYZ': -0.050479866172207837,
 'IIIIIYZZY': -0.0009000465964271142,
 'IIIIIZIII': 0.009

In [10]:
n_qubits = hamiltonians[mol_key][1]

ham_noncon = hamiltonians[mol_key][3]
Con_H = {}

for P in ham:
    if P in ham_noncon:
        continue
    else:
        Con_H[P]=ham[P]
        

## Testing contextuality

In [11]:
print('Is NONcontextual correct:', not c.contextualQ_ham(ham_noncon))
print('Is contextual correct:',c.contextualQ_ham(Con_H))

Is NONcontextual correct: True
Is contextual correct: True


# Classical part of problem!

Take $H_{nc}$ and split into:
- $Z$ = operators that completely comute with all operators in $S$
- $T$ = remaining operators in $S$
    - where $S = Z \cup T$  and $S$ is set of Pauli operators in $H_{nc}$
    
    
- We then split the set $T$ into cliques $C_{1}, C_{2}, ... , C_{|T|}$
    - all ops in a clique commute
    - ops between cliques anti-commute!

In [12]:
model = c.quasi_model(ham_noncon)
fn_form = c.energy_function_form(ham_noncon, model)

# Function form

$$R = G \cup \{ C_{i1} \;| \; i=1,2,...,N \}$$

- note q to do with $G$
- note r to do with $C_{i1}$

In [13]:
Energy_function = c.energy_function(fn_form)

In [14]:
ground_state_params = [list(hamiltonians[mol_key][5][1]), list(hamiltonians[mol_key][5][2])]
ground_state_params

[[1, 1, 1, 1, -1, 1, 1, -1], [1.7227299204006994e-08, -0.9999999999999999]]

In [15]:
## check

H_symbolic = conv_scr.Get_Openfermion_Hamiltonian(ham_noncon)
sparseH = qubit_operator_sparse(H_symbolic, n_qubits=n_qubits)

if sparseH.shape[0]<128:
    eig_vals, eig_vecs = sp.linalg.eigh(sparseH.todense())
    lowest_eigenvalue = min(eig_vals)
else:
    lowest_eigenvalue = sp.sparse.linalg.eigsh(sparseH, which='SA', k=1)[0][0]
    
np.isclose(Energy_function(*ground_state_params[0],*ground_state_params[1]),
           lowest_eigenvalue)

True

# Now need to rotate Hamiltonian!

We now have non contextual ground state: $(\vec{q}, \vec{r})$

In [16]:
ground_state_params

[[1, 1, 1, 1, -1, 1, 1, -1], [1.7227299204006994e-08, -0.9999999999999999]]

We can use this result - ground state of $H_{nc}$ -  as a classical estiamte of our ground state of the full Hamiltonian ($H = H_{c} + H_{nc}$)

However we can also obtain a quantum correction using $H_{c}$

By minimizing theenergy of the remaining terms in the Hamiltonian over the quantum states that are **consistent with the noncon-textual ground state**.

To do this we first rotate each $G_{j}$ and $\mathcal{A} = \sum_{i=1}^{N} r_{i}A_{i}$:

In [17]:
true_gs = hamiltonians[mol_key][4]
true_gs

-19.080086729695186

In [18]:
order = csvqe_results[mol_key][-1] # <-- order specified from greedy run by Will
qubit_removal_order=deepcopy(order)
qubit_removal_order

[4, 6, 5, 3, 0, 1, 2, 8, 7]

# Rotate full Hamiltonian to basis with diagonal noncontextual generators!

In [19]:
from cs_vqe_updated_up import diagonalize_epistemic_dictionary_generator

In [20]:
(mapping_GuA_to_singleZ_with_ep_exp_vals, 
 R_op_dict,
GuA_full_rotated) = diagonalize_epistemic_dictionary_generator(model, 
                                                            fn_form, 
                                                            ground_state_params, 
                                                            check_reduction=False,
                                                             up_method='LCU')
R_op_dict

{'unitary_part_method': 'LCU',
 'R_op': 0.7071067872773176 [] +
 0.7071067750957775j [X6 Y8]}

In [21]:
from cs_vqe_updated_up import get_wills_order

qubit_removal_order=deepcopy(order)

actual_qubit_removal_order = get_wills_order(GuA_full_rotated, qubit_removal_order)
actual_qubit_removal_order

[4, 6, 5, 3, 0, 1, 2, 8, 7]

In [22]:
mapping_GuA_to_singleZ_with_ep_exp_vals

{'ZIIIIIIII': {'single_Z': 'ZIIIIIIII',
  'noncon_gs_exp_val': 1,
  'do_unitary_part': False},
 'IZZIIIIII': {'single_Z': 'IZIIIIIII',
  'noncon_gs_exp_val': 1,
  'do_unitary_part': False},
 'IIZIIIIII': {'single_Z': 'IIZIIIIII',
  'noncon_gs_exp_val': 1,
  'do_unitary_part': False},
 'IIIZZIIII': {'single_Z': 'IIIZIIIII',
  'noncon_gs_exp_val': 1,
  'do_unitary_part': False},
 'IIIIZIZIZ': {'single_Z': 'IIIIZIIII',
  'noncon_gs_exp_val': -1,
  'do_unitary_part': False},
 'IIIIIZZZZ': {'single_Z': 'IIIIIZIII',
  'noncon_gs_exp_val': 1,
  'do_unitary_part': False},
 'IIIIIIZIZ': {'single_Z': 'IIIIIIZII',
  'noncon_gs_exp_val': 1,
  'do_unitary_part': False},
 'IIIIIIIZI': {'single_Z': 'IIIIIIIZI',
  'noncon_gs_exp_val': -1,
  'do_unitary_part': False},
 'IIIIIIXIX': {'single_Z': 'IIIIIIIIZ',
  'noncon_gs_exp_val': 1,
  'do_unitary_part': True}}

In [23]:
from cs_vqe_updated_up import diagonalize_epistemic_by_fixed_qubit

In [24]:
(rotations, 
 do_unitary_part, 
 ep_state_trans,
 GuA) = diagonalize_epistemic_by_fixed_qubit(model, mapping_GuA_to_singleZ_with_ep_exp_vals, [2])
GuA

['IIZIIIIII']

In [25]:
from cs_vqe_updated_up import get_reduced_hamiltonian_by_qubits_fixed

In [26]:
H_reduced = get_reduced_hamiltonian_by_qubits_fixed(ham,
                                             model, 
                                             mapping_GuA_to_singleZ_with_ep_exp_vals,
                                             [0],
                                             R_op_dict)
# H_reduced

In [27]:
from cs_vqe_updated_up import get_reduced_hamiltonians_by_order 


H_reduced_list = get_reduced_hamiltonians_by_order(ham,
                                             model, 
                                             mapping_GuA_to_singleZ_with_ep_exp_vals,
                                             actual_qubit_removal_order, # <-- actual GOOD order!
                                             R_op_dict)
H_reduced_list

[{'': (-19.046693391679273+0j)},
 {'I': (-18.44931121665712+0j),
  'Z': (0.5973821750221608+0j),
  'X': (-0.08321846052224319+0j)},
 {'II': (-17.441440227427314+0j),
  'IZ': (-1.0078709892298148+0j),
  'IX': (-0.03842233302047035+0j),
  'ZI': (0.9757113681001899+0j),
  'ZZ': (-0.3783291930780288+0j),
  'XX': (0.0826011041588578+0j),
  'XI': (-0.041609230261121594+0j),
  'YY': (-0.0826011041588578+0j),
  'XZ': (-0.041609230261121594+0j),
  'ZX': (0.03842233302047035+0j)},
 {'III': (-17.645935880586393+0j),
  'IIZ': (-0.5039354944617694+0j),
  'IYY': (0.03842233302047035+0j),
  'IZI': (0.5039354947680456+0j),
  'IZZ': (-0.2044956531590828+0j),
  'ZII': (0.7753772266145358+0j),
  'ZIZ': (-0.18916459653901452+0j),
  'ZXZ': (-0.0024777800235672824+0j),
  'ZZI': (0.18916459653901452+0j),
  'ZZX': (-0.00247777968692792+0j),
  'XYY': (-0.04130055207942889+0j),
  'XZZ': (0.020804615130560797+0j),
  'YYI': (0.023873272021881158-0j),
  'YYX': (-0.0413005520794289+0j),
  'YIY': 0.02387327202188115

In [28]:
qubit_removal_order=deepcopy(order)
qubit_removal_order
print(qubit_removal_order)
print()

red_Hs = c.get_reduced_hamiltonians(ham,
                           model,
                           fn_form,
                           ground_state_params,
                           qubit_removal_order)
red_Hs

[4, 6, 5, 3, 0, 1, 2, 8, 7]



[{'': -19.046693391679266},
 {'I': -18.449311216657115, 'Z': -0.597382175022161, 'X': 0.08321846052224319},
 {'II': -17.441440227011963,
  'IZ': 1.0078709896451605,
  'YY': 0.038422333035975725,
  'ZZ': 0.9757113682298244,
  'ZI': 0.37832919320766367,
  'ZX': -0.08260110409477918,
  'XZ': -0.041609230261121594,
  'IX': 0.08260110409477917,
  'XI': 0.041609230261121594,
  'XX': 0.038422333035975725},
 {'III': -17.64593588020116,
  'IIZ': 0.5039354948469996,
  'YIY': 0.038422333035975725,
  'IZZ': -0.5039354947981608,
  'IZI': -0.20449565318919816,
  'ZIZ': 0.7753772267162188,
  'ZII': 0.1891645966406974,
  'ZXZ': -0.0024777800526880854,
  'ZZI': -0.18916459656696608,
  'XXX': 0.0024777800526476317,
  'ZIX': -0.04130055201535029,
  'XZZ': 0.020804615130560797,
  'YYI': 0.023873272021881158,
  'IIX': 0.04130055207942889,
  'IYY': 0.023873272021881158,
  'IZX': -0.04130055201535029,
  'XIZ': -0.020804615130560797,
  'ZYY': -0.023873272008198446,
  'XXI': -0.023873272021881158,
  'ZZX': 0.0

In [29]:
len(H_reduced_list) == len(red_Hs)

True

In [30]:
E_list_one =[]
for H in red_Hs:
    H = {P_key: coeff.real for P_key, coeff in H.items() if not np.isclose(coeff.real,0)}
    
    H_mat = conv_scr.Get_Openfermion_Hamiltonian(H)
    sparseH = qubit_operator_sparse(H_mat, n_qubits=hamiltonians[mol_key][1])
    E = sp.sparse.linalg.eigsh(sparseH, which='SA', k=1)[0][0]
#     print(E)
#     print(len(H))
#     print()
    E_list_one.append(E)

In [31]:
E_list_two =[]
for H in H_reduced_list:
    H = {P_key: coeff.real for P_key, coeff in H.items() if not np.isclose(coeff.real,0)}
    H_mat = conv_scr.Get_Openfermion_Hamiltonian(H)
    sparseH = qubit_operator_sparse(H_mat, n_qubits=hamiltonians[mol_key][1])
    E = sp.sparse.linalg.eigsh(sparseH, which='SA', k=1)[0][0]
#     print(E)
#     print(len(H))
#     print()
    E_list_two.append(E)

In [32]:
np.array(E_list_two) - np.array(E_list_one)

array([-7.10542736e-15, -3.55271368e-15,  2.47268872e-12,  7.26529947e-12,
        4.85300689e-12,  9.94759830e-13, -1.27116095e-11, -2.61088928e-11,
        1.45661261e-13,  1.99413819e-11])

In [33]:
# cs_VQE new
np.array(E_list_two) - true_gs

array([ 3.33933380e-02,  2.76248062e-02,  2.42046870e-02,  2.41211479e-02,
        1.71639736e-02,  5.76291411e-03,  2.99193132e-03,  3.36158237e-04,
        2.28638102e-04, -4.61852778e-14])

In [34]:
# cs_VQE will
np.array(E_list_one) - true_gs

array([ 3.33933380e-02,  2.76248062e-02,  2.42046870e-02,  2.41211479e-02,
        1.71639736e-02,  5.76291411e-03,  2.99193133e-03,  3.36158263e-04,
        2.28638102e-04, -1.99875672e-11])

In [35]:
# look for: do_unitary_part=TRUE
mapping_GuA_to_singleZ_with_ep_exp_vals


{'ZIIIIIIII': {'single_Z': 'ZIIIIIIII',
  'noncon_gs_exp_val': 1,
  'do_unitary_part': False},
 'IZZIIIIII': {'single_Z': 'IZIIIIIII',
  'noncon_gs_exp_val': 1,
  'do_unitary_part': False},
 'IIZIIIIII': {'single_Z': 'IIZIIIIII',
  'noncon_gs_exp_val': 1,
  'do_unitary_part': False},
 'IIIZZIIII': {'single_Z': 'IIIZIIIII',
  'noncon_gs_exp_val': 1,
  'do_unitary_part': False},
 'IIIIZIZIZ': {'single_Z': 'IIIIZIIII',
  'noncon_gs_exp_val': -1,
  'do_unitary_part': False},
 'IIIIIZZZZ': {'single_Z': 'IIIIIZIII',
  'noncon_gs_exp_val': 1,
  'do_unitary_part': False},
 'IIIIIIZIZ': {'single_Z': 'IIIIIIZII',
  'noncon_gs_exp_val': 1,
  'do_unitary_part': False},
 'IIIIIIIZI': {'single_Z': 'IIIIIIIZI',
  'noncon_gs_exp_val': -1,
  'do_unitary_part': False},
 'IIIIIIXIX': {'single_Z': 'IIIIIIIIZ',
  'noncon_gs_exp_val': 1,
  'do_unitary_part': True}}

In [40]:
for Z_key in mapping_GuA_to_singleZ_with_ep_exp_vals:
    if mapping_GuA_to_singleZ_with_ep_exp_vals[Z_key]['do_unitary_part'] is True:
        Z_term = mapping_GuA_to_singleZ_with_ep_exp_vals[Z_key]['single_Z']
        print('qubit ind where unitary part is done:', Z_term.index('Z'))

qubit ind where unitary part is done: 8


In [42]:
for Z_key in mapping_GuA_to_singleZ_with_ep_exp_vals:
    print(Z_key)

ZIIIIIIII
IZZIIIIII
IIZIIIIII
IIIZZIIII
IIIIZIZIZ
IIIIIZZZZ
IIIIIIZIZ
IIIIIIIZI
IIIIIIXIX


In [43]:
actual_qubit_removal_order

[4, 6, 5, 3, 0, 1, 2, 8, 7]

In [ ]:
order

In [ ]:
order

In [44]:
from cs_vqe_updated_up import get_reduced_H_minimal_rotations

In [45]:
order = deepcopy(csvqe_results[mol_key][-1]) # <-- order specified from greedy run by Will

H_reduced_seqrot_csvqe = get_reduced_H_minimal_rotations(ham,
                                             model, 
                                             fn_form,
                                             ground_state_params,
                                             order,
                                             check_reduction=True,
                                             up_method='SeqRot')


In [49]:
order = deepcopy(csvqe_results[mol_key][-1]) # <-- order specified from greedy run by Will

H_reduced_LCU_csvqe = get_reduced_H_minimal_rotations(ham,
                                             model, 
                                             fn_form,
                                             ground_state_params,
                                             order,
                                             check_reduction=True,
                                             up_method='LCU')

In [50]:
for ind, H_LCU in enumerate(H_reduced_LCU_csvqe['H_csvqe']):
    print(len(H_LCU))
    print(len(H_reduced_seqrot_csvqe['H_csvqe'][ind]))
    print()

1
1

3
3

10
10

34
34

71
71

132
132

216
216

268
268

412
412

596
596



In [51]:
hamiltonians[mol_key][4]

-19.080086729695186

In [53]:
csvqe_results[mol_key][2]

[0.03339333801600475,
 0.027624806161423265,
 0.02420468699649092,
 0.024121147942604892,
 0.017163973571538094,
 0.005762914110160722,
 0.0029919313340656117,
 0.0003361582629004545,
 0.00022863810193385348,
 -1.98632221781736e-11]

In [52]:
csvqe_results[mol_key][1]

[-53.248970562330506,
 -53.248970562330534,
 -53.25754457502269,
 -19.05596558175257,
 -19.062922756123637,
 -53.268651926948884,
 -53.27376966756567,
 -53.28056799105157,
 -53.288184741834975,
 -53.32722736882277]

In [54]:
mol_num = 26
molecules_and_qubits = [(mol_key, hamiltonians[mol_key][1])for mol_key in csvqe_results]
sorted_by_qubit_No = sorted(molecules_and_qubits, key= lambda x: x[1])
mol_key = sorted_by_qubit_No[mol_num-1][0] # UCL supercomputer indexes from 1, hence minus one here!

print(f'{mol_key: <25} n_qubits: {hamiltonians[mol_key][1]}')

H2-S1_STO-3G_singlet      n_qubits: 18


In [55]:
from quchem.Unitary_Partitioning.Graph import Clique_cover_Hamiltonian
from quchem.Unitary_Partitioning.Unitary_partitioning_Seq_Rot import SeqRot_linalg_Energy, SeqRot_linalg_Energy_NEW

In [56]:
# optional params!
commutativity_flag = 'AC' ## <- defines relationship between sets!!!
Graph_colouring_strategy='largest_first'
check_reduction_LCU = False
prune_threshold = 1e-6


### with measurement reduction
m_red_seqrot_with_seqrot_cs_vqe = {}
for ind_key, H_seqrot in enumerate(H_reduced_seqrot_csvqe['H_csvqe']):
    if ind_key == 0:
        # only non-contextual problem
        m_red_seqrot_with_seqrot_cs_vqe[ind_key] = {'AC_sets': {}, 'ground_state': None}
    else:
        ### SeqRot
        H_SeqRot_pruned = {P_key: coeff.real for P_key, coeff in H_seqrot.items() if
                           np.abs(coeff) > prune_threshold}

        H_SeqRot = conv_scr.Get_Openfermion_Hamiltonian(H_SeqRot_pruned)
        n_qubits = len(list(H_seqrot.keys())[0])

        anti_commuting_sets_SeqRot = Clique_cover_Hamiltonian(H_SeqRot,
                                                              n_qubits,
                                                              commutativity_flag,
                                                              Graph_colouring_strategy)
        
        m_red_seqrot_with_seqrot_cs_vqe[ind_key] = {'AC_sets': anti_commuting_sets_SeqRot}

m_red_seqrot_with_seqrot_cs_vqe

{0: {'AC_sets': {}, 'ground_state': None},
 1: {'AC_sets': {0: [-18.449311216963398 []],
   1: [0.5973821750221608 [Z0], -0.08321846052224319 [X0]]}},
 2: {'AC_sets': {0: [-17.441440227427314 []],
   1: [-1.0078709895360907 [Z1],
    -0.03842233302047035 [X1],
    -0.0826011041588578 [Y0 Y1]],
   2: [0.9757113681001899 [Z0],
    0.0826011041588578 [X0 X1],
    -0.041609230261121594 [X0 Z1]],
   3: [-0.3783291930780288 [Z0 Z1],
    -0.041609230261121594 [X0],
    0.03842233302047035 [Z0 X1]]}},
 3: {'AC_sets': {0: [-17.645935880586393 []],
   1: [-0.5039354947680452 [Z2],
    -0.0024777800235672755 [Z0 Z1 X2],
    -0.04130055207942889 [X0 Y1 Y2],
    -0.023873272021881154 [X0 X2]],
   2: [0.5039354947680456 [Z1],
    -0.0024777800235672824 [Z0 X1 Z2],
    -0.0413005520794289 [Y0 Y1 X2],
    0.023873272021881158 [X0 X1 Z2]],
   3: [-0.18916459653901452 [Z0 Z2],
    0.0024777800235672807 [Z1 X2],
    0.020804615130560797 [X0 Z1 Z2],
    0.023873272021881158 [X0 X1]],
   4: [0.189164596539

In [ ]:
m_red_seqrot_with_seqrot_cs_vqe.keys()

In [87]:
ind = 9
S_key_dict = dict((zip(list(m_red_seqrot_with_seqrot_cs_vqe[ind]['AC_sets'].keys()),
                  [0 for _ in range(len(m_red_seqrot_with_seqrot_cs_vqe[ind]['AC_sets'].keys()))])))
# S_key_dict
N_Qubits = len(list(H_reduced_seqrot_csvqe['H_csvqe'][ind].keys())[0])
N_Qubits

9

In [63]:
m_red_seqrot_with_seqrot_cs_vqe[ind]

{'AC_sets': {0: [-11.893640687855415 []],
  1: [-0.2931388759906617 [Z2],
   0.012124689610585604 [X1 X2],
   0.0037664248486789656 [Y2 X3 Y4],
   -0.003459006871821896 [Y2 Y3 Z5 Z6 Z7 Z8]],
  2: [-0.2931388759906621 [Z1],
   0.012124689610585604 [Y1 Y2],
   -0.003459006871821896 [X1 X3 Z5 Z6 Z7 Z8],
   -0.0037664248486789656 [X1 Y3 Y4]],
  3: [0.3892355930337432 [Z1 Z2],
   -0.003459006871821896 [X2 X3 Z5 Z6 Z7 Z8],
   -0.0037664248486789656 [X2 Y3 Y4],
   0.0037664248486789656 [Y1 Y3 X4],
   0.003459006871821896 [X0 Y2 Y4 Z5 Z6 Z7 Z8]],
  4: [-0.020667907053551374 [Z4],
   -0.011350949894639856 [Z3 X4 Z5 Z6 Z7 Z8],
   0.0037664248486789656 [X2 X3 X4],
   0.003459006871821896 [X0 Y1 Y4 Z5 Z6 Z7 Z8],
   -0.003459006871821896 [Y0 X2 Y4 Z5 Z6 Z7 Z8],
   0.004425472648664972 [Z0 Y4 Z5 Y6 X8],
   0.003541551921219104 [Z0 Z3 X4 X8],
   -0.020538305079828578 [Z0 Z3 X4 X6 Z7 Z8]],
  5: [-0.020667907053551374 [Z3],
   0.011350949894639842 [X0 X3 Z4 Z5 Z6 Z7 Z8],
   0.0037664248486789656 [X1 X3

In [85]:
HH = m_red_seqrot_with_seqrot_cs_vqe[ind]['AC_sets'][0]


-11.893640687855415

In [67]:
SeqRot_linalg_Energy(m_red_seqrot_with_seqrot_cs_vqe[ind]['AC_sets'],
                     S_key_dict,
                     N_Qubits, 
                     atol=1e-8, 
                     rtol=1e-05,
                     check_reduction=False)

-19.0800867296952

In [66]:
SeqRot_linalg_Energy_NEW(m_red_seqrot_with_seqrot_cs_vqe[ind]['AC_sets'],
                     S_key_dict,
                     N_Qubits, 
                     atol=1e-8, 
                     rtol=1e-05,
                     check_reduction=False)

-19.080086729695182